# Custom Layers

## 1. Layers without Parameters

In [1]:
import torch
from torch import nn
from torch.nn import functional as F
from d2l import torch as d2l

class CenteredLayer(nn.Module):
    def __init__(self):
        super().__init__()

    def forward(self, X):
        return X - X.mean()

In [2]:
layer = CenteredLayer()
layer(torch.tensor([1.0, 2, 3, 4, 5]))

tensor([-2., -1.,  0.,  1.,  2.])

In [3]:
net = nn.Sequential(nn.LazyLinear(128), CenteredLayer())

/home/wangzixiao/miniconda3/envs/d2l/lib/python3.9/site-packages/torch/nn/modules/lazy.py:178: UserWarning: Lazy modules are a new feature under heavy development so changes to the API or functionality can happen at any moment.
  warnings.warn('Lazy modules are a new feature under heavy development '


In [4]:
# Check the layer. The output should have mean 0
Y = net(torch.rand(4, 8))
Y.mean()

tensor(1.0245e-08, grad_fn=<MeanBackward0>)

## 2. Layers with Parameters

Previously we use `nn.Linear` and it is a layer with parameters. We can also use built-in functions to create parameters, which provide some basic housekeeping functionality. In particular, they govern access, initialization, sharing, saving, and loading model parameters. This way, among other benefits, we will not need to write custom serialization routines for every custom layer.

In [22]:
class MyLinear(nn.Module):
    def __init__(self, in_units, units):
        super().__init__()
        self.weight = nn.Parameter(torch.randn(in_units, units))
        self.bias = nn.Parameter(torch.randn(units, ))
    
    def forward(self, X):
        linear = torch.matmul(X, self.weight.data) + self.bias.data
        return F.relu(linear)

In [23]:
linear = MyLinear(5, 3)
linear.weight

Parameter containing:
tensor([[ 1.3143,  1.4855,  0.3992],
        [-1.3025,  0.2687, -0.7280],
        [ 0.3896,  1.2831,  0.5261],
        [ 1.1223,  0.7525,  2.0211],
        [-0.5470, -0.3401,  0.5112]], requires_grad=True)

In [24]:
linear(torch.randn(2, 5))

tensor([[0.0000, 0.0000, 0.0000],
        [1.2679, 0.2810, 0.0000]])

In [25]:
net = nn.Sequential(MyLinear(64, 8), MyLinear(8, 1))
net(torch.randn(2, 64))

tensor([[8.6318],
        [5.9052]])